Note: I left the code to generate the htfs for each test system in the notebook, but if you don't want to re-generate, you can use the pickled htf that I saved (I left the full path in for you). I generated these htfs based after you added support for `lambda_{i}_bonds_old/new` and `scale_lambda_0_bonds`

In [1]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools
from perses.tests.test_topology_proposal import generate_dipeptide_top_pos_sys, generate_atp
import pickle
import copy

#############################################
# CONSTANTS
#############################################
temperature = 298.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CUDA")


conducting subsequent work with the following platform: CUDA


INFO:rdkit:Enabling RDKit 2021.03.1 jupyter extensions


In [2]:
def test_angle_energies(htf, is_old=True, is_solvated=False, check_scale=False):
    htf_copy = copy.deepcopy(htf)
    
    # Get harmonic angle force and old/new positions
    system = htf._topology_proposal.old_system if is_old else htf._topology_proposal.new_system
    harmonic_angle_force = system.getForce(1) 
    positions = htf.old_positions(htf.hybrid_positions) if is_old else htf.new_positions(htf.hybrid_positions)
    
    # Get custom angle force and hybrid positions
    angle_force_index = 2 if is_solvated else 1
    hybrid_system = htf.hybrid_system
    custom_angle_force = hybrid_system.getForce(angle_force_index)
    hybrid_positions = htf.hybrid_positions
    
    # Remove nonbonded/exception forces
    for i in range(hybrid_system.getNumForces(), hybrid_system.getNumForces() - 4, -1):
        hybrid_system.removeForce(i - 1)
    
    # Set global parameters
    lambda_old = 1 if is_old else 0
    lambda_new = 0 if is_old else 1
    for i in range(custom_angle_force.getNumGlobalParameters()):
        if custom_angle_force.getGlobalParameterName(i) == 'lambda_0_angles_old':
            custom_angle_force.setGlobalParameterDefaultValue(i, lambda_old)
        if custom_angle_force.getGlobalParameterName(i) == 'lambda_0_angles_new':
            custom_angle_force.setGlobalParameterDefaultValue(i, lambda_new)

    # Zero the unique old/new angles in the custom bond force
    hybrid_to_angle_indices = htf._hybrid_to_new_angle_indices if is_old else htf._hybrid_to_old_angle_indices
    for hybrid_idx, idx in hybrid_to_angle_indices.items():
        p1, p2, p3, hybrid_params = custom_angle_force.getAngleParameters(hybrid_idx)
        hybrid_params = list(hybrid_params)
        index_to_zero = -3 if is_old else -1
        hybrid_params[index_to_zero] *= 0
        custom_angle_force.setAngleParameters(hybrid_idx, p1, p2, p3, hybrid_params)

    ## Get energy components of standard bond force
    platform = configure_platform(REFERENCE_PLATFORM)
    thermostate_other = ThermodynamicState(system=system, temperature=temperature)
    integrator_other = openmm.VerletIntegrator(1.0*unit.femtosecond)
    context_other = thermostate_other.create_context(integrator_other)
    context_other.setPositions(positions)
    components_other = compute_potential_components(context_other, beta=beta)
    print(components_other)
    
    ## Get energy components of custom bond force
    if is_solvated:
        custom_angle_force.setUsesPeriodicBoundaryConditions(True)
    thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
    integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
    context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
    context_hybrid.setPositions(hybrid_positions)
    components_hybrid = compute_potential_components(context_hybrid, beta=beta)
    print(components_hybrid)
    
    assert np.isclose([components_other[1][1]], [components_hybrid[1][1]])
    
    print("Success! Custom angle force and standard angle force energies are equal!")
    
    if check_scale:
        
        # Get custom bond force and hybrid positions
        angle_force_index = 2 if is_solvated else 1
        hybrid_system = htf_copy.hybrid_system
        custom_angle_force = hybrid_system.getForce(angle_force_index)
        hybrid_positions = htf_copy.hybrid_positions
        
        if is_solvated:
            custom_angle_force.setUsesPeriodicBoundaryConditions(True)
        
        # Remove nonbonded/exception forces
        for i in range(hybrid_system.getNumForces(), hybrid_system.getNumForces() - 4, -1):
            hybrid_system.removeForce(i - 1)
        
        ## Get energy components of custom bond force
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid)
        
        # Set `scale_lambda_{i}` to 0.5
        for i in range(custom_angle_force.getNumGlobalParameters()):
            if custom_angle_force.getGlobalParameterName(i) == 'scale_lambda_0_angles':
                custom_angle_force.setGlobalParameterDefaultValue(i, 0.5)
            elif custom_angle_force.getGlobalParameterName(i) == 'interscale_lambda_0_angles':
                custom_angle_force.setGlobalParameterDefaultValue(i, 0.5)
        
        ## Get energy components of custom bond force iwth scaling
        thermostate_hybrid = ThermodynamicState(system=hybrid_system, temperature=temperature)
        integrator_hybrid = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context_hybrid = thermostate_hybrid.create_context(integrator_hybrid)
        context_hybrid.setPositions(hybrid_positions)
        components_hybrid_scaled = compute_potential_components(context_hybrid, beta=beta)
        print(components_hybrid_scaled)
        
        assert not np.isclose([components_hybrid[1][1]], [components_hybrid_scaled[1][1]])
        
        print("Success! Scaling the angle force changes the energy")

# Alanine dipeptide in vacuum

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [3]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [5]:
test_angle_energies(htf)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356520876), ('HarmonicAngleForce', 0.6112775462128573), ('PeriodicTorsionForce', 16.285398289983085), ('NonbondedForce', -39.449927745600164), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.296256769802195), ('CustomAngleForce', 0.6112775462128995), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [6]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum.pickle", "rb") as f:
    htf = pickle.load(f)

In [7]:
test_angle_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 1.2957330745733426), ('HarmonicAngleForce', 3.403930427792299), ('PeriodicTorsionForce', 30.268405909483896), ('NonbondedForce', 388.51299936325097), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1.296256769802195), ('CustomAngleForce', 3.403930427792298), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one scale region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [8]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [10]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 22, 23, 24, 25, 26, 27, 28, 29]

In [11]:
test_angle_energies(htf)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.03478365356521381), ('HarmonicAngleForce', 0.6112775462128995), ('PeriodicTorsionForce', 16.285398289983064), ('NonbondedForce', -39.4499277456003), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 3.0809325282327777), ('CustomAngleForce', 0.6112775462128995), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [12]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_vacuum_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [13]:
test_angle_energies(htf, is_old=False)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 3.0804088330039257), ('HarmonicAngleForce', 12.337771305429378), ('PeriodicTorsionForce', 28.091644511264406), ('NonbondedForce', 56.74765841438245), ('AndersenThermostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 3.0809325282327777), ('CustomAngleForce', 12.337771305429378), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


# Alanine dipeptide in solvent

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [14]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [15]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034783653565214685), ('HarmonicAngleForce', 0.6112775462129015), ('PeriodicTorsionForce', 16.285398289983082), ('NonbondedForce', -1017.9419702202149), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.108758935146324), ('CustomAngleForce', 0.6112775462128995), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [16]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [17]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 2.108235239917542), ('HarmonicAngleForce', 8.824888133365281), ('PeriodicTorsionForce', 25.495955289982533), ('NonbondedForce', -1054.9212696042337), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 2.108758935146324), ('CustomAngleForce', 8.824888133365286), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one scale region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [18]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [19]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[10, 11, 12, 13, 1557, 1558, 1559, 1560, 1561, 1562, 1563, 1564]

In [20]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.034783653565214685), ('HarmonicAngleForce', 0.6112775462129015), ('PeriodicTorsionForce', 16.285398289983082), ('NonbondedForce', -995.1493033223735), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.915410547041641), ('CustomAngleForce', 0.6112775462128995), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [3]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [4]:
test_angle_energies(htf, is_old=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.9148885029095102), ('HarmonicAngleForce', 9.207577639387514), ('PeriodicTorsionForce', 27.87597844994209), ('NonbondedForce', -940.6739740442471), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.915410547041641), ('CustomAngleForce', 9.207653737542774), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.915410547041641), ('CustomAngleForce', 9.661801162668427), ('CustomTorsionForce', 16.285403285921724), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.91

# CDK2 transformation in solvent

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [5]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [6]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.608082977954756), ('HarmonicAngleForce', 29.199523609209102), ('PeriodicTorsionForce', 15.076747992617829), ('NonbondedForce', 3350.162471334583), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 29.199523609209102), ('CustomTorsionForce', 15.076753795301011), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [7]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [8]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 4.519508183899664), ('HarmonicAngleForce', 31.40442991986144), ('PeriodicTorsionForce', 15.078153431480851), ('NonbondedForce', 3363.615071727043), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 4.608082977954756), ('CustomAngleForce', 31.404429919861446), ('CustomTorsionForce', 15.076753795301011), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one scale region

In [3]:
from openeye import oechem

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [4]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_regions.pickle", "rb") as f:
    htf = pickle.load(f)

ModuleNotFoundError: No module named 'oechem'

In [ ]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

In [ ]:
test_angle_energies(htf, is_solvated=True)

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [ ]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/cdk2_solvent_scale_regions.pickle", "rb") as f:
    htf = pickle.load(f)

In [ ]:
test_angle_energies(htf, is_old=False, is_solvated=True, check_scale=True)

# 8mer in solvent

### Test one alchemical region, no scale regions

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [5]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)

In [6]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.349099420286866), ('HarmonicAngleForce', 12.529213143558692), ('PeriodicTorsionForce', 157.29212321868366), ('NonbondedForce', -10413.634803293226), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 8.88521073611831), ('CustomAngleForce', 12.529213143555554), ('CustomTorsionForce', 157.2921325778617), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [7]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent.pickle", "rb") as f:
    htf = pickle.load(f)


In [8]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 8.839279898124524), ('HarmonicAngleForce', 382.3538632504383), ('PeriodicTorsionForce', 168.47244240570083), ('NonbondedForce', -6861.17431642076), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 8.88521073611831), ('CustomAngleForce', 382.3538632504382), ('CustomTorsionForce', 157.2921325778617), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, one scale region

#### Old system HarmonicBondForce vs hybrid system CustomBondForce

In [9]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [10]:
# This is the scale region
list(htf._atom_classes['unique_old_atoms'][0]) + list(htf._atom_classes['unique_new_atoms'][0])

[24, 25, 26, 21, 5575, 5576, 5577, 5578, 5579]

In [11]:
test_angle_energies(htf, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 7.349099420288431), ('HarmonicAngleForce', 12.529213143555554), ('PeriodicTorsionForce', 157.292123218685), ('NonbondedForce', -10511.125832891092), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12.665182674760329), ('CustomAngleForce', 12.529213143555554), ('CustomTorsionForce', 157.2921325778617), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12.665182674760329), ('CustomAngleForce', 222.15532081099008), ('CustomTorsionForce', 157.2921325778617), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12

#### New system HarmonicBondForce vs hybrid system CustomBondForce

In [12]:
# Load htf
with open("/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/code/26_rxn_field/8mer_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [13]:
test_angle_energies(htf, is_old=False, is_solvated=True, check_scale=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 12.619251836766544), ('HarmonicAngleForce', 222.03488526154752), ('PeriodicTorsionForce', 165.54093146134787), ('NonbondedForce', 80580.58719803792), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12.665182674760329), ('CustomAngleForce', 222.03488526154752), ('CustomTorsionForce', 157.2921325778617), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 12.665182674760329), ('CustomAngleForce', 222.15532081099008), ('CustomTorsionForce', 157.2921325778617), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 1

## Barnase:barstar (old htf, no naked charge fix or counterion fix)

### Test one alchemical region, no scale regions

#### Old system HarmonicAngleForce vs hybrid system CustomAngleForce

In [14]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [15]:
test_angle_energies(htf, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 190.96991399580065), ('HarmonicAngleForce', 797.0922707362635), ('PeriodicTorsionForce', 3856.2346108037004), ('NonbondedForce', -23496.968598492353), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.92874049962995), ('CustomAngleForce', 797.0922707362635), ('CustomTorsionForce', 3856.2347805821146), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


#### New system HarmonicAngleForce vs hybrid system CustomAngleForce

In [16]:
# Load htf
with open("T42A_complex.pickle", "rb") as f:
    htf = pickle.load(f)

In [17]:
test_angle_energies(htf, is_old=False, is_solvated=True)

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 191.51760393383896), ('HarmonicAngleForce', 1010.4766887798822), ('PeriodicTorsionForce', 3849.6843670339917), ('NonbondedForce', -22772.6842316537), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 191.92874049962995), ('CustomAngleForce', 1010.4766887798822), ('CustomTorsionForce', 3856.2347805821146), ('AndersenThermostat', 0.0), ('MonteCarloBarostat', 0.0)]
Success! Custom angle force and standard angle force energies are equal!


### Test one alchemical region, no scale regions

#### Old system HarmonicAngleForce vs hybrid system CustomAngleForce

In [18]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

ModuleNotFoundError: No module named 'oechem'

In [ ]:
test_angle_energies(htf, is_solvated=True, check_scale=True)

#### New system HarmonicAngleForce vs hybrid system CustomAngleForce

In [ ]:
# Load htf
with open("T42A_complex_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)

In [ ]:
test_angle_energies(htf, is_old=False, is_solvated=True, check_scale=True)

In [ ]:
# TO DO: actually test the scaling of the energies matches